In [88]:
import json
import requests
import pandas as pd
from pandas import json_normalize
import numpy as np
from datetime import datetime
from tqdm import tqdm
# Set display options to show all columns
pd.set_option('display.max_columns', None)

In [89]:
# Load headers from JSON file
with open("../secrets/api_headers.json", "r") as file:
    headers = json.load(file)

# # API endpoint
# url = "https://myanimelist.p.rapidapi.com/anime/top/airing"

# # Make a GET request to the API
# response = requests.get(url, headers=headers)

# # Check if the request was successful
# if response.status_code == 200:
#     # Parse JSON response
#     api_data = response.json()

#     # Extract desired fields from the API response
#     extracted_data = []
#     for entry in api_data:
#         anime_data = {
#             "date_pulled": datetime.now().strftime('%Y-%m-%d'),  # Record the date when data is pulled
#             "myanimelist_id": entry["myanimelist_id"],
#             "title": entry["title"],
#             "rank": entry["rank"],
#             "score": entry["score"]
#         }
#         extracted_data.append(anime_data)

#     # Write the extracted data to a CSV file
#     csv_file = "../data/top_airing.csv"
#     with open(csv_file, mode='w', newline='') as file:
#         fieldnames = ["date_pulled", "myanimelist_id", "title", "rank", "score"]
#         writer = csv.DictWriter(file, fieldnames=fieldnames)
        
#         # Write the header row
#         writer.writeheader()
        
#         # Write the data rows
#         writer.writerows(extracted_data)

#     print(f"Data saved to {csv_file}")

# else:
#     print("Failed to fetch data from API")


In [105]:
current_top_50 = pd.read_csv('../data/top_airing.csv')
current_top_50.head(10)

,date_pulled,myanimelist_id,title,rank,score
0,2023-10-11,51535,Shingeki no Kyojin: The Final Season - Kankets...,1,9.02
1,2023-10-11,52991,Sousou no Frieren,2,8.94
2,2023-10-11,51009,Jujutsu Kaisen 2nd Season,3,8.83
3,2023-10-11,54595,Kage no Jitsuryokusha ni Naritakute! 2nd Season,4,8.75
4,2023-10-11,21,One Piece,5,8.71
5,2023-10-11,53887,Spy x Family Season 2,6,8.45
6,2023-10-11,44042,Holo no Graffiti,7,8.22
7,2023-10-11,54714,Kimi no Koto ga Daidaidaidaidaisuki na 100-nin...,8,8.19
8,2023-10-11,235,Detective Conan,9,8.17
9,2023-10-11,54112,Zom 100: Zombie ni Naru made ni Shitai 100 no ...,10,8.08


In [106]:
# Anime table
anime = pd.read_csv('../data/anime.csv', usecols=['myanimelist_id'])
# Your initial data (assuming you have it in a DataFrame named 'df')
api_url = "https://myanimelist.p.rapidapi.com/anime/"

# New DataFrame to store flattened data
flattened_data_df = pd.DataFrame()

# Iterate through myanimelist_ids in the initial data
for myanimelist_id in tqdm(current_top_50['myanimelist_id'].head(10)):
    if myanimelist_id not in anime['myanimelist_id'].values:
        # Make API request for detailed anime information
        response = requests.get(f'{api_url}{myanimelist_id}', headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse JSON response
            api_data = response.json()

            # Flatten the nested JSON structure
            flattened_data = json_normalize(api_data)

            # Add 'myanimelist_id' as an additional column
            flattened_data['myanimelist_id'] = myanimelist_id

            # Append flattened data to the existing DataFrame
            flattened_data_df = pd.concat([flattened_data_df, flattened_data])

        else:
            print(f"Failed to fetch data for myanimelist_id: {myanimelist_id}")

# Save the updated DataFrame to CSV file (append mode)
flattened_data_df.to_csv("../data/new_anime.csv", index=False)

print("Flattened data appended to new_anime.csv")


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:10<00:00,  1.06s/it]

Flattened data appended to new_anime.csv


In [110]:
new = pd.read_csv('../data/new_anime.csv')
old = pd.read_csv('../data/anime.csv')

combined = pd.concat([old, new], ignore_index=True)
combined.to_csv('../data/anime.csv')

In [111]:
old = pd.read_csv('../data/anime.csv')
old

,Unnamed: 0,picture_url,title_en,characters,synopsis,title_ov,alternative_titles.english,alternative_titles.synonyms,alternative_titles.japanese,information.studios,information.aired,information.source,information.status,information.producers,information.genres,information.genre,information.premiered,information.licensors,information.theme,information.broadcast,information.demographic,information.episodes,information.rating,information.duration,information.themes,information.type,statistics.popularity,statistics.favorites,statistics.score,statistics.members,statistics.ranked,myanimelist_id,data,alternative_titles.german
0,0,https://cdn.myanimelist.net/images/anime/1279/...,Attack on Titan: The Final Season - Final Chap...,[{'myanimelist_url': 'https://myanimelist.net/...,In the wake of Eren Yeager's cataclysmic actio...,Shingeki no Kyojin: The Final Season - Kankets...,Attack on Titan: The Final Season - Final Chap...,"Shingeki no Kyojin: The Final Season Part 3, S...",進撃の巨人 The Final Season完結編,[{'url': 'https://myanimelist.net/anime/produc...,"Mar 4, 2023 to Nov 5, 2023",Manga,Currently Airing,[{'url': 'https://myanimelist.net/anime/produc...,[{'url': 'https://myanimelist.net/anime/genre/...,NaN,NaN,NaN,NaN,NaN,[{'url': 'https://myanimelist.net/anime/genre/...,2,R - 17+ (violence & profanity),1 hr. 1 min. per ep.,[{'url': 'https://myanimelist.net/anime/genre/...,[{'url': 'https://myanimelist.nethttps://myani...,436.0,10202.0,9.02,486634.0,12.0,51535,NaN,NaN
1,1,https://cdn.myanimelist.net/images/anime/1015/...,Frieren: Beyond Journey's End,[{'myanimelist_url': 'https://myanimelist.net/...,"The demon king has been defeated, and the vict...",Sousou no Frieren,Frieren: Beyond Journey's End,Frieren at the Funeral,葬送のフリーレン,[{'url': 'https://myanimelist.net/anime/produc...,"Sep 29, 2023 to ?",Manga,Currently Airing,[{'url': 'https://myanimelist.net/anime/produc...,[{'url': 'https://myanimelist.net/anime/genre/...,NaN,[{'url': 'https://myanimelist.nethttps://myani...,NaN,NaN,Fridays at 23:00 (JST),[{'url': 'https://myanimelist.net/anime/genre/...,Unknown,PG-13 - Teens 13 or older,25 min.,NaN,[{'url': 'https://myanimelist.nethttps://myani...,1261.0,1774.0,8.94,180705.0,17.0,52991,NaN,NaN
2,2,https://cdn.myanimelist.net/images/anime/1792/...,Jujutsu Kaisen Season 2,[{'myanimelist_url': 'https://myanimelist.net/...,"The year is 2006, and the halls of Tokyo Prefe...",Jujutsu Kaisen 2nd Season,Jujutsu Kaisen Season 2,"Jujutsu Kaisen: Kaigyoku Gyokusetsu, Jujutsu K...",呪術廻戦 懐玉・玉折／渋谷事変,[{'url': 'https://myanimelist.net/anime/produc...,"Jul 6, 2023 to ?",Manga,Currently Airing,[{'url': 'https://myanimelist.net/anime/produc...,[{'url': 'https://myanimelist.net/anime/genre/...,NaN,[{'url': 'https://myanimelist.nethttps://myani...,NaN,[{'url': 'https://myanimelist.net/anime/genre/...,Thursdays at 23:56 (JST),[{'url': 'https://myanimelist.net/anime/genre/...,23,R - 17+ (violence & profanity),23 min. per ep.,NaN,[{'url': 'https://myanimelist.nethttps://myani...,331.0,9861.0,8.83,616611.0,28.0,51009,NaN,NaN
3,3,https://cdn.myanimelist.net/images/anime/1938/...,The Eminence in Shadow Season 2,[{'myanimelist_url': 'https://myanimelist.net/...,"Everything has been going according to plan, b...",Kage no Jitsuryokusha ni Naritakute! 2nd Season,The Eminence in Shadow Season 2,Shadow Garden 2nd Season,陰の実力者になりたくて！ 2nd Season,[{'url': 'https://myanimelist.net/anime/produc...,"Oct 4, 2023 to ?",Light novel,Currently Airing,[{'url': 'https://myanimelist.net/anime/produc...,[{'url': 'https://myanimelist.net/anime/genre/...,NaN,[{'url': 'https://myanimelist.nethttps://myani...,[{'url': 'https://myanimelist.net/anime/produc...,NaN,Wednesdays at 22:30 (JST),NaN,12,R - 17+ (violence & profanity),23 min. per ep.,[{'url': 'https://myanimelist.net/anime/genre/...,[{'url': 'https://myanimelist.nethttps://myani...,1224.0,1952.0,8.75,185994.0,46.0,54595,NaN,NaN
4,4,https://cdn.myanimelist.net/images/anime/6/732...,NaN,[{'myanimelist_url': 'https://myanimelist.net/...